In [ ]:
"""
from google.colab import drive
drive.mount('/content/gdrive')
"""

In [11]:
#imports
#@title Import & seed
import time
import re
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import requests
import collections
import pickle
import copy, random
import nltk as nl
nl.download('punkt')
from itertools import zip_longest

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input, Reshape, BatchNormalization, Dense, Dropout, concatenate,
    Embedding, LSTM, Dense, GRU, Bidirectional, Add
)
from tensorflow.keras.activations import elu, relu, softmax, sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical

vocab_size = 1900
terces_per_batch = 8
terces_len = 200
batch_len = terces_per_batch * (terces_len + 1)
EPOCHS = 30


#Setup
print(tf.__version__)

!nvidia-smi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2.6.0
Thu Sep 23 11:16:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    58W / 149W |   1192MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------------------

In [2]:
# title Setup wandb
!pip install wandb
!wandb login e6569c556c797b5bed38bc6cba40d24b68e8973d
import wandb
wandb.init(project="DeepComedy", name="bananito96")

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 139 kB 45.1 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 180 kB 45.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=6c0c07cc224a51bfb9dbd333665075ae4046c008ddfc46df2354c58d7f847c18
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=8dfcf92f0604b79228c5201ef3283e59ee9cf4e50dc8d4d49fe9a3ffee5b125b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: dropino (use `wandb login --relogin` to force relogin)


#Clean files


In [ ]:
"""
#count chars
char_list = []
with open(FILENAME) as file:
  while True:
    char = file.read(1)
    if not char:
      print("End of file")
      break
    char = char.lower()

    #add good char to char list
    if char not in char_list:
      char_list.append(char)

print(char_list)
"""

In [ ]:
"""
removable_chars = ['•', '—', '-', '(', ')', ',', '.', ':', ';', '“', '”', '«', '»', '"','0','1','2','3','4','5','6','7','8','9']
#‘ e ’ usate per racchiudere parti parlate in latino
#“ e ” racchiude il parlato sia in italiano che latino
#idem « e »


clean_char_list = []
removedCharsCounter = 0
fileOut = open(CLEANFILENAME, "a")

with open(FILENAME) as file:
  while True:
    char = file.read(1)
    if not char:
      print("End of file")
      break

    #transform upper case to lower case
    char = char.lower()

    #remove unwanted chars
    if char not in removable_chars:

      #add acceptable chars to char list
      if char not in clean_char_list:
        clean_char_list.append(char)
      
      #output in new file
      fileOut.write(char)
    else:
      removedCharsCounter += 1


print(char_list)
print(removedCharsCounter)
"""

In [ ]:
"""
f = open(CLEANFILENAME, "r")
lines = f.readlines()

number_of_syllables = []
line_counter = 1

for line in lines:
  #excludes all empty lines and canto intros
  if '|' in line:
    syllables = line.count('|')
    number_of_syllables.append(syllables)
    if syllables != 11:
      print("line number {} with text {} has {} syllables".format(line_counter,line,syllables))
    line_counter += 1

Counter(number_of_syllables)
"""

# Tokenization

"""Once tokenized the dataset, a map of the syllables with the integer index is created. The final dimension
of the vocabulary is 1874 tokens but it is limited to 1800 to remove the tail of infrequent syllables.

We substituted every space between words with the special token < SEP > and inserted at the beginning
of each verse the token < GO >. To make all verses the same lengths, we used the special character
“< PAD >” to pad every terces to the length of 75 tokens. At the end of each verse we appended the
symbol “< EOV > “, while at the end of each sentence “< EOS >”."""

number_of_syllables = []
line_counter = 1
"""
space -> <SPA>
verst start -> <VST>
padding (up to 75) -> <PAD>
end of verse -> <EOV>
end of sentence -> <EOS>
"""


In [3]:
def get_hyp_lm_tercets(tercets):
    new_tercets = []
    for tercet in tercets:
        new_tercets.append([])
        for verse in tercet:
            new_tercets[-1].append([])
            for hyp_w in verse:
                new_tercets[-1][-1].extend(hyp_w)
                new_tercets[-1][-1].append('<SEP>')
            new_tercets[-1][-1] = new_tercets[-1][-1][:-1]

    return new_tercets

def hyphenation(verse):
    """
    Split word in syllables
    :param verse: input string
    :return: a list containing syllables of the word
    """
    syllables = []
    syllable = ""

    verse = verse[1:]
    verse = verse+"|" 

    for letter in verse:
        syllable += letter
        if letter == '|':
            syllables.append(syllable)
            syllable = ""
               
    return syllables

def get_dc_hyphenation(canti):
    hyp_canti, hyp_tokens = [], []
    for canto in canti:
        hyp_canti.append([])
        for verso in canto:
            syllables = hyphenation(verso)
            hyp_canti[-1].append(syllables)
            for syllable in syllables:
                hyp_tokens.extend(syllable)

    return hyp_canti, hyp_tokens
    

def get_dc_cantos(filename, encoding=None):
    cantos = []
    cantoCounter =  0
    cantos.append([])
    with open(filename, "r", encoding=encoding) as f:
        for line in f:
            sentence = line.strip()
            if 'canto' not in sentence and len(sentence) > 2:
                if len(sentence) > 2:
                    cantos[cantoCounter].append(sentence)
    return cantos


def create_tercets(cantos):
    tercets = []
    for i,canto in enumerate(cantos):
        for v,verse in enumerate(canto):
            if v%3 == 0:
                tercets.append([])

            tercets[-1].append(verse)
        tercets = tercets[:-1]  # removes the last malformed tercets (only 2 verses)

    return tercets


def pad_list(l, pad_token, max_l_size, keep_lasts=False, pad_right=True):
    """
    Adds a padding token to a list
    inputs:
    :param l: input list to pad.
    :param pad_token: value to add as padding.
    :param max_l_size: length of the new padded list to return,
    it truncates lists longer that 'max_l_size' without adding
    padding values.
    :param keep_lasts: If True, preserves the max_l_size last elements
    of a sequence (by keeping the same order).  E.g.:
    if keep_lasts is True and max_l_size=3 [1,2,3,4] becomes [2,3,4].


    :return: the list padded or truncated.
    """
    to_pad = []
    max_l = min(max_l_size, len(l))  # maximum len
    l_init = len(l) - max_l if len(l) > max_l and keep_lasts else 0  # initial position where to sample from the list
    l_end = len(l) if len(l) > max_l and keep_lasts else max_l
    for i in range(l_init, l_end):
        to_pad.append(l[i])

    # for j in range(len(l), max_l_size):
    #     to_pad.append(pad_token)
    pad_tokens = [pad_token] * (max_l_size-len(l))
    padded_l = to_pad + pad_tokens if pad_right else pad_tokens + to_pad

    return padded_l


class Vocabulary(object):
    def __init__(self, vocab_size=None):
        self.dictionary = dict()
        self.rev_dictionary = dict()
        self.count = []
        self.special_tokens = []
        self.vocab_size = vocab_size

    def build_vocabulary_from_counts(self, count, special_tokens=[]):
        """
        Sets all the attributes of the Vocabulary object.
        :param count: a list of lists as follows: [['token', number_of_occurrences],...]
        :param special_tokens: a list of strings. E.g. ['<EOS>', '<PAD>',...]
        :return: None
        """

        dictionary = dict()
        for word, _ in count:
            dictionary[word] = len(dictionary)

        # adding eventual special tokens to the dictionary (e.g. <EOS>,<PAD> etc..)
        d = len(dictionary)
        for i, token in enumerate(special_tokens):
            dictionary[token] = d + i

        self.count = count
        self.dictionary = dictionary
        self.rev_dictionary = dict(zip(self.dictionary.values(), self.dictionary.keys()))
        self.special_tokens = special_tokens
        self.vocab_size = len(dictionary)

    def build_vocabulary_from_tokens(self, tokens, vocabulary_size=None, special_tokens=[]):
        """
        Given a list of tokens, it sets the Vocabulary object attributes by constructing
        a dictionary mapping each token to a unique id.
        :param tokens: a list of strings.
         E.g. ["the", "cat", "is", ... ".", "the", "house" ,"is" ...].
         NB: Here you should put all your token instances of the corpus.
        :param vocabulary_size: The number of elements of your vocabulary. If there are more
        than 'vocabulary_size' elements on tokens, it considers only the 'vocabulary_size'
        most frequent ones.
        :param special_tokens: Optional. A list of strings. Useful to add special tokens in vocabulary.
        If you don't have any, keep it empty.
        :return: None
        """

        vocabulary_size = vocabulary_size if vocabulary_size is not None else self.vocab_size
        vocabulary_size = vocabulary_size - (len(special_tokens) + 1) if vocabulary_size else None
        # counts occurrences of each token
        count = [['<UNK>', -1]]
        count.extend(collections.Counter(tokens).most_common(vocabulary_size))  # takes only the most frequent ones, if size is None takes them all
        self.build_vocabulary_from_counts(count, special_tokens)  # actually build the vocabulary
        self._set_unk_count(tokens)  # set the number of OOV instances

    def string2id(self, dataset):
        """
        Converts a dataset of strings into a dataset of ids according to the object dictionary.
        :param dataset: any string-based dataset with any nested lists.
        :return: a new dataset, with the same shape of dataset, where each string is mapped into its
        corresponding id associated in the dictionary (0 for unknown tokens).
        """

        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, str) or isinstance(item, int) or isinstance(item, float):
                    new_items.append(self.word2id(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def id2string(self, dataset):
        """
        Converts a dataset of integer ids into a dataset of string according to the reverse dictionary.
        :param dataset: any int-based dataset with any nested lists. Allowed types are int, np.int32, np.int64.
        :return: a new dataset, with the same shape of dataset, where each token is mapped into its
        corresponding string associated in the reverse dictionary.
        """
        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, int) or isinstance(item, np.int) or isinstance(item, np.int32) or isinstance(item, np.int64):
                    new_items.append(self.id2word(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def word2id(self, item):
        """
        Maps a string token to its corresponding id.
        :param item: a string.
        :return: If the token belongs to the vocabulary, it returns an integer id > 0, otherwise
        it returns the value associated to the unknown symbol, that is typically 0.
        """
        return self.dictionary[item] if item in self.dictionary else self.dictionary['<UNK>']

    def id2word(self, token_id):
        """
        Maps an integer token to its corresponding string.
        :param token_id: an integer.
        :return: If the id belongs to the vocabulary, it returns the string
        associated to it, otherwise it returns the string associated
        to the unknown symbol, that is '<UNK>'.
        """

        return self.rev_dictionary[token_id] if token_id in self.rev_dictionary else self.rev_dictionary[self.dictionary['<UNK>']]

    def _set_unk_count(self, tokens):
        """
        Sets the number of OOV instances in the tokens provided
        :param tokens: a list of tokens
        :return: None
        """
        data = list()
        unk_count = 0
        for word in tokens:
            if word in self.dictionary:
                index = self.dictionary[word]
            else:
                index = 0  # dictionary['<UNK>']
                unk_count += 1
            data.append(index)
        self.count[0][1] = unk_count

    def add_element(self, name, is_special_token=False):
        if name not in self.dictionary:
            self.vocab_size += 1
            self.dictionary[name] = self.vocab_size
            self.rev_dictionary[self.vocab_size] = name

            if is_special_token:
                self.special_tokens = list(self.special_tokens)
                self.special_tokens.append(name)

            self.count.append([name, 1])

    def set_vocabulary(self, dictionary, rev_dictionary, special_tokens, vocab_size):
        self.dictionary = dictionary,
        self.rev_dictionary = rev_dictionary
        self.special_tokens = special_tokens
        self.vocab_size = vocab_size


class SyLMDataset(object):
    def __init__(self, config, sy_vocab=None):
        self.config = config
        self.vocabulary = sy_vocab

        self.raw_train_x = []
        self.raw_val_x = []
        self.raw_test_x = []
        self.raw_x = []

        self.train_x, self.train_y = [], []
        self.val_x, self.val_y = [], []
        self.test_x, self.test_y = [], []
        self.x, self.y = [], []

    def initialize(self, sess):
        pass

    def load(self, sources):
        """
        Extract raw texts form sources and gather them all together.
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :return: a list of raw strings.
        """
        return NotImplementedError

    def build(self, sources, split_size=0.8):
        """
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :param split_size: the size to split the dataset, set >=1.0 to not split.
        """

        raw_x = self.load(sources)
        # raw_x = self.tokenize([self.preprocess(ex) for ex in raw_x])  # fixme
        # splitting data
        self.raw_x = raw_x
        if split_size < 1.0:
            self.raw_train_x, self.raw_test_x = self.split(self.raw_x, train_size=split_size)
            self.raw_train_x, self.raw_val_x = self.split(self.raw_train_x, train_size=split_size)
        else:
            self.raw_train_x = self.raw_x

        if self.vocabulary is None:
            # creates vocabulary
            tokens = [item for sublist in self.raw_train_x for item in sublist]  # get tokens
            special_tokens = ("<GO>", "<PAD>", "<SEP>", "<EOS>", "<EOV>")
            self._create_vocab(tokens, special_tokens=special_tokens)

        # creates x,y for train
        self.train_x = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.train_y = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for validation
        self.val_x = self._build_dataset(self.raw_val_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.val_y = self._build_dataset(self.raw_val_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for testing
        self.test_x = self._build_dataset(self.raw_test_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.test_y = self._build_dataset(self.raw_test_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

    def _create_vocab(self, tokens, special_tokens=("<PAD>", "<GO>", "<SEP>", "<EOV>", "<EOS>")):
        """
        Create the vocabulary. Special tokens can be added to the tokens obtained from
        the corpus.
        :param tokens: a list of all the tokens in the corpus. Each token is a string.
        :param special_tokens: a list of strings.
        """
        print("creating_vocabulary") 

        vocab = Vocabulary(vocab_size=self.config.input_vocab_size)
        vocab.build_vocabulary_from_tokens(tokens, special_tokens=special_tokens)
        self.vocabulary = vocab

    @staticmethod
    def split(raw_data, train_size=0.8):
        size = math.floor(len(raw_data)*train_size)
        return raw_data[:size], raw_data[size:]

    @staticmethod
    def preprocess(txt):
        return txt

    @staticmethod
    def shuffle(x):
        return random.sample(x, len(x))

    @staticmethod
    def tokenize(txt):
        return txt

    def _build_dataset(self, raw_data, max_len=200, insert_go=True, keep_lasts=False, pad_right=True, shuffle=True):
        """
        Converts all the tokens in e1_raw_data by mapping each token with its corresponding
        value in the dictionary. In case of token not in the dictionary, they are assigned to
        a specific id. Each sequence is padded up to the seq_max_len setup in the config.

        :param raw_data: list of sequences, each sequence is a list of tokens (strings).
        :param max_len: max length of a sequence, crop longer and pad smaller ones.
        :param insert_go: True to insert <GO>, False otherwise.
        :param keep_lasts: True to truncate initial elements of a sequence.
        :param pad_right: pad to the right (default value True), otherwise pads to left.
        :param shuffle: Optional. If True data are shuffled.
        :return: A list of sequences where each token in each sequence is an int id.
        """
        dataset = []
        for sentence in raw_data:
            sentence_ids = [self.vocabulary.word2id("<GO>")] if insert_go else []
            sentence_ids.extend([self.vocabulary.word2id(w) for w in sentence])
            sentence_ids.append(self.vocabulary.word2id("<EOS>"))
            sentence_ids = pad_list(sentence_ids, self.vocabulary.word2id("<PAD>"), max_len, keep_lasts=keep_lasts, pad_right=pad_right)

            dataset.append(sentence_ids)

        if shuffle:
            return random.sample(dataset, len(dataset))
        else:
            return dataset

    def get_batches(self, batch_size=32, split_sel='train'):
        """
        Iterator over the training set. Useful method to run experiments.
        :param batch_size: size of the mini_batch
        :return: input and target.
        """
        if split_sel == 'train':
            x, y = self.train_x, self.train_y
        elif split_sel == 'val':
            x, y = self.val_x, self.val_y
        else:
            x, y = self.test_x, self.test_y
        
        i = 0 #random.randint(0, batch_size)
        batches = []
        eov = self.vocabulary.word2id("<EOV>")
        go = self.vocabulary.word2id("<GO>")
        # prepare batches
        while i < len(x):
            j = 0
            batch_x, batch_y = [], []
            while j < batch_size and i+j<len(x):
                for c in x[i+j]:
                  batch_x.append(c)
                batch_x.append(eov)
                for c in y[i+j]:
                  batch_y.append(c)
                batch_y.append(eov)
                j += 1
            i += batch_size
            batches.append((batch_x, batch_y))

        # supply
        i = 0
        while i < len(batches):
            yield batches[i][0], batches[i][1]
            i += 1

class DanteSyLMDataset(SyLMDataset):
    def __init__(self, config, sy_vocab=None):
        """
        Class to create a dataset from Dante Alighieri's Divine Comedy.
        :param config: a Config object
        :param sy_vocab: (optional) a Vocabulary object where tokens of the dictionary
        are syllables. If None, the vocabulary is create automatically from the source.
        """
        super().__init__(config, sy_vocab)

    def load(self, sources):
        """
        Load examples from dataset
        :param sources: data filepath.
        :return:
        """
        canti = get_dc_cantos(filename=sources)  # get raw data from file
        canti, tokens = get_dc_hyphenation(canti)  # converts each

        tercets = create_tercets(canti)
        tercets = get_hyp_lm_tercets(tercets)
        x = []
        for tercet in tercets:
            x.append([])
            for verse in tercet:
                x[-1].extend(verse)
                x[-1].append("<EOV>")

        #x = self.shuffle(x)
        return x

def seq2str(seq):
    def output2string(batch, rev_vocabulary, special_tokens, end_of_tokens):
        to_print = ''
        for token in batch:
            if token in special_tokens:
                to_print += ' '
            elif end_of_tokens and token in end_of_tokens:
                to_print += '\n'
            elif token in rev_vocabulary:
                to_print += rev_vocabulary[token]
            else:
                to_print += '<UNK>'
        return to_print

    return output2string(seq, poetry_sy_lm_dataset.vocabulary.rev_dictionary,
      special_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<PAD>"), 
                      0, 
                      poetry_sy_lm_dataset.vocabulary.word2id("<SEP>"),
                      poetry_sy_lm_dataset.vocabulary.word2id("<GO>"), 
                      poetry_sy_lm_dataset.vocabulary.word2id("<EOS>")],
      end_of_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<EOV>")])

class cnfg:
  vocab_size = vocab_size
  input_vocab_size = vocab_size
  sentence_max_len = terces_len

config = cnfg()
poetry_sy_lm_dataset = DanteSyLMDataset(config, sy_vocab=None)

data_path = 'syllComedy.txt'  # dataset location, here just the name of the source file

poetry_sy_lm_dataset.build(data_path, split_size=0.99)  # actual creation of  vocabulary (if not provided) and dataset
print("Train size: " + str(len(poetry_sy_lm_dataset.train_y)))
print("Val size: " + str(len(poetry_sy_lm_dataset.val_y)))
print("Test size: " + str(len(poetry_sy_lm_dataset.test_y)))

eov = poetry_sy_lm_dataset.vocabulary.word2id("<EOV>")
pad = poetry_sy_lm_dataset.vocabulary.word2id("<PAD>")
go = poetry_sy_lm_dataset.vocabulary.word2id("<GO>")
eos = poetry_sy_lm_dataset.vocabulary.word2id("<EOS>")


batches = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch)]
print(batches[0][0])
print(len(batches[0][0]))
test_b = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch, split_sel='test')]
print(len(test_b[0][0]))
val_b = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch, split_sel='val')]
print(len(val_b[0][0]))
len(poetry_sy_lm_dataset.vocabulary.dictionary.items())

creating_vocabulary
Train size: 4649
Val size: 47
Test size: 48
[40, 8, 4, 10, 3, 1, 42, 17, 4, 26, 1, 42, 26, 7, 3, 1, 42, 14, 4, 10, 3, 1, 42, 13, 5, 17, 1, 42, 17, 6, 8, 3, 1, 42, 14, 6, 3, 1, 42, 8, 7, 1, 42, 12, 11, 9, 5, 3, 1, 42, 19, 6, 1, 42, 11, 5, 1, 44, 17, 6, 3, 1, 42, 9, 6, 1, 42, 11, 9, 7, 1, 42, 19, 5, 6, 3, 1, 42, 18, 4, 9, 3, 1, 42, 16, 1, 42, 8, 5, 3, 1, 42, 12, 4, 10, 1, 42, 19, 5, 3, 7, 1, 42, 12, 13, 16, 1, 42, 9, 5, 1, 44, 13, 22, 31, 3, 1, 42, 10, 5, 3, 1, 42, 14, 6, 1, 42, 9, 6, 11, 1, 42, 11, 5, 3, 1, 42, 19, 6, 5, 3, 1, 42, 4, 1, 42, 9, 5, 3, 1, 42, 12, 17, 5, 9, 1, 42, 9, 6, 1, 42, 11, 5, 1, 44, 43, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 44, 40, 5, 22, 6, 3, 1, 42, 24, 16, 5, 8, 1, 42, 11, 7, 3, 5, 3, 1, 42, 14, 6, 9, 3, 1, 42, 24, 16, 5, 10, 3, 1, 42, 4, 1, 42, 9, 5, 3, 30, 3, 1, 42, 13, 7, 1, 42, 12, 5, 3, 1, 42, 14, 16, 1, 42, 9, 5, 1, 44, 4, 1, 42, 12, 11, 5, 

43

In [4]:
poetry_sy_lm_dataset.vocabulary.dictionary

{' ': 3,
 '!': 34,
 '<EOS>': 43,
 '<EOV>': 44,
 '<GO>': 40,
 '<PAD>': 41,
 '<SEP>': 42,
 '<UNK>': 0,
 '?': 33,
 'a': 5,
 'b': 25,
 'c': 13,
 'd': 14,
 'e': 4,
 'f': 23,
 'g': 21,
 'h': 22,
 'i': 6,
 'j': 38,
 'l': 10,
 'm': 17,
 'n': 8,
 'o': 7,
 'p': 18,
 'q': 24,
 'r': 9,
 's': 12,
 't': 11,
 'u': 16,
 'v': 19,
 'x': 37,
 'y': 39,
 'z': 26,
 '|': 1,
 'à': 32,
 'è': 30,
 'é': 31,
 'ì': 27,
 'ò': 29,
 'ó': 35,
 'ù': 28,
 '‘': 36,
 '’': 20}

In [ ]:
print(seq2str(batches[14][0]))
print(seq2str(batches[15][0]))
print(seq2str(batches[16][0]))


 vi| di e | co| nob| bi | l’ om| bra | di | co| lui|
che | fe| ce | per | vil| ta| de il | gran | ri| fiu| to|
in| con| ta| nen| te in| te| si e | cer| to | fui|
                                      
 che | que| sta e| ra | la | set| ta | d’ i | cat| ti| vi|
a | dio | spia| cen| ti e | a’ | ne| mi| ci | sui|
que| sti | sciau| ra| ti | che | mai | non | fur | vi| vi|
                               
 e| ra| no i| gnu| di e | sti| mo| la| ti | mol| to|
da | mo| sco| ni e | da | ve| spe | ch’ e| ran | i| vi|
el| le | ri| ga| van | lor | di | san| gue il | vol| to|
                                  
 che | mi| schia| to | di | la| gri| me a’ | lor | pie| di|
da | fa| sti| dio| si | ver| mi e| ra | ri| col| to|
e | poi | ch’ a | ri| guar| dar | ol| tre | mi | die| di|
                             
 vi| di | gen| ti a | la | ri| va | d’ un | gran | fiu| me|
per | ch’ io | dis| si | ma| e| stro or | mi | con| ce| di|
ch’ i’ | sap| pia | qua| li | so| no e | qual | co| stu| me|
               

# Model

In [12]:
wandb.config.num_layers = 4
wandb.config.d_model = 128
wandb.config.dff = 256
wandb.config.num_heads = 4
wandb.config.dropout = 0.1
wandb.config.learning_rate = 2e-4 

generate_at = [] #[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

def create_padding_mask(seq):   
    seq = tf.cast(tf.math.equal(seq, pad), tf.float32)
    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.
    
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.
        
    Returns:
        output, attention_weights
    """
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  
    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)
    return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)
            
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
        
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = tf.reshape(scaled_attention, 
                                    (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        return output, attention_weights

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def __call__(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
        
        return out2

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)
    
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
        
        
    def __call__(self, x, enc_output, training, 
            look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
        
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
        
        ffn_output = self.ffn(out2)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
        
        return out3, attn_weights_block1, attn_weights_block2

class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                        for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
            
    def __call__(self, x, training, mask):
        seq_len = tf.shape(x)[1]
        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        return x  # (batch_size, input_seq_len, d_model)

class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
                maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                        for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
        
    def __call__(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}
        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                look_ahead_mask, padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
        
        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
                target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                            input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                            target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
        
    def __call__(self, inp, tar, training, enc_padding_mask, 
            look_ahead_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
        
        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)
        
        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        
        return final_output, attention_weights

optimizer = tf.keras.optimizers.Adam(wandb.config.learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='train_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

input_vocab_size = vocab_size
target_vocab_size = vocab_size
max_len = batch_len

transformer = Transformer(wandb.config.num_layers, wandb.config.d_model, 
                          wandb.config.num_heads, wandb.config.dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=max_len, 
                          pe_target=max_len,
                          rate=wandb.config.dropout)
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)
    
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)
    
    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    
    return enc_padding_mask, combined_mask, dec_padding_mask

checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

@tf.function()
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    
    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp, 
                                    True, 
                                    enc_padding_mask, 
                                    combined_mask, 
                                    dec_padding_mask)
        loss = loss_function(tar_real, predictions)
        gradients = tape.gradient(loss, transformer.trainable_variables)    
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    
    train_loss(loss)
    train_accuracy(tar_real, predictions)

@tf.function()
def val_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    
    predictions, _ = transformer(inp, tar_inp, 
                                False, 
                                enc_padding_mask, 
                                combined_mask, 
                                dec_padding_mask)
    loss = loss_function(tar_real, predictions)
    
    val_loss(loss)
    val_accuracy(tar_real, predictions)

Latest checkpoint restored!!


In [13]:
#@title Generation
def generate(index=0, k=1, t=1):

    def evaluate_greedy(inp_sentence, decoder_input):
        inp_sentence = inp_sentence
        encoder_input = tf.expand_dims(inp_sentence, 0)
        
        output = tf.expand_dims(decoder_input, 0)

        terces = 0
        for i in range(batch_len):
            enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
                encoder_input, output)
        
            # predictions.shape == (batch_size, seq_len, vocab_size)
            predictions, attention_weights = transformer(encoder_input, 
                                                        output,
                                                        False,
                                                        enc_padding_mask,
                                                        combined_mask,
                                                        dec_padding_mask)
            
            # select the last word from the seq_len dimension
            predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

            predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

            # return the result if the predicted_id is equal to the end token
            if predicted_id == eos:
                terces += 1
                if terces == terces_per_batch-1:
                    return tf.squeeze(output, axis=0), attention_weights
            # concatentate the predicted_id to the output which is given to the decoder
            # as its input.
            output = tf.concat([output, predicted_id], axis=-1)

        return tf.squeeze(output, axis=0), attention_weights


    def evaluate_topk(inp_sentence, decoder_input, k=5, temperature=0.5):
        inp_sentence = inp_sentence
        encoder_input = tf.expand_dims(inp_sentence, 0)
        
        output = tf.expand_dims(decoder_input, 0)

        def scale(tensor):
            tensor = tf.math.divide(
                tf.subtract(
                    tensor, 
                    tf.reduce_min(tensor)
                ), 
                tf.subtract(
                    tf.reduce_max(tensor), 
                    tf.reduce_min(tensor))
                )
            return tensor

        terces = 0
        for i in range(batch_len):
            enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
                encoder_input, output)
        
            # predictions.shape == (batch_size, seq_len, vocab_size)
            predictions, attention_weights = transformer(encoder_input, 
                                                        output,
                                                        False,
                                                        enc_padding_mask,
                                                        combined_mask,
                                                        dec_padding_mask)
            # select the last word from the seq_len dimension
            predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
            predictions, indices = tf.math.top_k(predictions,k=k)
            predictions /= temperature
            #predictions = scale(predictions)
            predictions = np.squeeze(predictions, axis=0)
            indices = np.squeeze(indices, axis=0)
            indices = np.squeeze(indices, axis=0)
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            predicted_id = indices[predicted_id]

            # return the result if the predicted_id is equal to the end token
            if predicted_id == eos:
                terces += 1
                if terces == terces_per_batch-1:
                    return tf.squeeze(output, axis=0), attention_weights
            # concatentate the predicted_id to the output which is given to the decoder
            # as its input.
            predicted_id = tf.expand_dims(predicted_id, 0)
            predicted_id = tf.expand_dims(predicted_id, 0)
            output = tf.concat([output, predicted_id], axis=-1)

        return tf.squeeze(output, axis=0), attention_weights

    out_list = test_b[index][0]
    offset = terces_len # a tercet
    txt_gen = seq2str(out_list[-offset:])

    print("params: k={}, t={}".format(k,t))
    for i in range(32//(terces_per_batch-1)): # 30 terces = cantica
        out, att_w = evaluate_topk([pad], out_list[-offset:], k, t)
        out_list = out.numpy().tolist()
        out_str = seq2str(out_list[offset:])
        txt_gen += out_str

    print(txt_gen)
    wandb.log({"generated":
            wandb.Html("k="+str(k)+" t="+str(t)+
                       "<pre>"+txt_gen+"</pre>", inject=False)})

# Train Loop

In [14]:
#@title Train loop

for epoch in range(EPOCHS):
    start_time = time.time()
    random.shuffle(batches)
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (inp, tar)) in enumerate(batches):
        if (len(inp) != batch_len or len(tar) != batch_len):
            print("discarded batch", batch)
            continue
        train_step(np.expand_dims(inp, axis=0), np.expand_dims(tar, axis=0))
        
        if batch % 50 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                epoch + 1, batch, train_loss.result(), train_accuracy.result()))
        
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, train_loss.result(), train_accuracy.result()))
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

    wandb.log({
        'train_loss': train_loss.result(),
        'train_accuracy': train_accuracy.result()
    }, step=epoch+1)

    # validation
    if epoch % 5 == 0:
        loss_l, acc_l = [], []
        for (batch, (inp, tar)) in enumerate(val_b):
            val_loss.reset_states()
            val_accuracy.reset_states()
            
            if (len(inp) != batch_len or len(tar) != batch_len):
                print("discarded batch", batch)
                continue

            val_step(np.expand_dims(inp, axis=0), np.expand_dims(tar, axis=0))

            loss_l.append(val_loss.result())
            acc_l.append(val_accuracy.result())

        loss_mean = sum(loss_l)/len(loss_l)
        acc_mean = sum(acc_l)/len(acc_l)
        print('Epoch {} VALIDATION: Loss {:.4f} Accuracy {:.4f}\n'.format(epoch + 1, loss_mean, acc_mean))

        wandb.log({
            'val_loss': loss_mean,
            'val_accuracy': acc_mean
        }, step=epoch+1)

    # generation
    if epoch in generate_at:
        generate(1)
    
    print("epoch lasted: {}".format(time.time()-start_time))

Epoch 1 Batch 0 Loss 1.1158 Accuracy 0.6192
Epoch 1 Batch 50 Loss 1.0749 Accuracy 0.6304
Epoch 1 Batch 100 Loss 1.0739 Accuracy 0.6317
Epoch 1 Batch 150 Loss 1.0740 Accuracy 0.6319
discarded batch 184
Epoch 1 Batch 200 Loss 1.0742 Accuracy 0.6313
Epoch 1 Batch 250 Loss 1.0731 Accuracy 0.6317
Epoch 1 Batch 300 Loss 1.0721 Accuracy 0.6316
Epoch 1 Batch 350 Loss 1.0719 Accuracy 0.6319
Epoch 1 Batch 400 Loss 1.0719 Accuracy 0.6319
Epoch 1 Batch 450 Loss 1.0712 Accuracy 0.6323
Epoch 1 Batch 500 Loss 1.0709 Accuracy 0.6324
Epoch 1 Batch 550 Loss 1.0698 Accuracy 0.6324


wandb: WARNING Step must only increase in log calls.  Step 1 < 21; dropping {'train_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.070079>, 'train_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.63223183>}.


Epoch 1 Loss 1.0701 Accuracy 0.6322
Time taken for 1 epoch: 122.40270566940308 secs



wandb: WARNING Step must only increase in log calls.  Step 1 < 21; dropping {'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.0209386>, 'val_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.64741755>}.


discarded batch 5
Epoch 1 VALIDATION: Loss 1.0209 Accuracy 0.6474

epoch lasted: 124.76849341392517
Epoch 2 Batch 0 Loss 1.0774 Accuracy 0.6397
Epoch 2 Batch 50 Loss 1.0667 Accuracy 0.6343
Epoch 2 Batch 100 Loss 1.0663 Accuracy 0.6338
Epoch 2 Batch 150 Loss 1.0628 Accuracy 0.6351
Epoch 2 Batch 200 Loss 1.0630 Accuracy 0.6345
Epoch 2 Batch 250 Loss 1.0625 Accuracy 0.6346
Epoch 2 Batch 300 Loss 1.0616 Accuracy 0.6350
Epoch 2 Batch 350 Loss 1.0618 Accuracy 0.6347
Epoch 2 Batch 400 Loss 1.0613 Accuracy 0.6352
Epoch 2 Batch 450 Loss 1.0615 Accuracy 0.6353
discarded batch 459
Epoch 2 Batch 500 Loss 1.0606 Accuracy 0.6355
Epoch 2 Batch 550 Loss 1.0603 Accuracy 0.6356


...

Epoch 18 Loss 0.9636 Accuracy 0.6688
Time taken for 1 epoch: 111.23070430755615 secs

epoch lasted: 111.23532557487488
Epoch 19 Batch 0 Loss 0.9415 Accuracy 0.6926
discarded batch 7
Epoch 19 Batch 50 Loss 0.9616 Accuracy 0.6712
Epoch 19 Batch 100 Loss 0.9603 Accuracy 0.6714
Epoch 19 Batch 150 Loss 0.9599 Accuracy 0.6711
Epoch 19 Batch 200 Loss 0.9597 Accuracy 0.6704
Epoch 19 Batch 250 Loss 0.9590 Accuracy 0.6708
Epoch 19 Batch 300 Loss 0.9585 Accuracy 0.6708
Epoch 19 Batch 350 Loss 0.9583 Accuracy 0.6711
Epoch 19 Batch 400 Loss 0.9590 Accuracy 0.6709
Epoch 19 Batch 450 Loss 0.9594 Accuracy 0.6708
Epoch 19 Batch 500 Loss 0.9592 Accuracy 0.6709
Epoch 19 Batch 550 Loss 0.9586 Accuracy 0.6713


wandb: WARNING Step must only increase in log calls.  Step 19 < 21; dropping {'train_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.95821905>, 'train_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.67143637>}.


Epoch 19 Loss 0.9582 Accuracy 0.6714
Time taken for 1 epoch: 111.24335265159607 secs

epoch lasted: 111.24864912033081
Epoch 20 Batch 0 Loss 0.9670 Accuracy 0.6783
Epoch 20 Batch 50 Loss 0.9560 Accuracy 0.6740
Epoch 20 Batch 100 Loss 0.9562 Accuracy 0.6720
Epoch 20 Batch 150 Loss 0.9528 Accuracy 0.6728
Epoch 20 Batch 200 Loss 0.9523 Accuracy 0.6729
Epoch 20 Batch 250 Loss 0.9530 Accuracy 0.6730
Epoch 20 Batch 300 Loss 0.9534 Accuracy 0.6729
Epoch 20 Batch 350 Loss 0.9541 Accuracy 0.6728
discarded batch 364
Epoch 20 Batch 400 Loss 0.9541 Accuracy 0.6729
Epoch 20 Batch 450 Loss 0.9535 Accuracy 0.6732
Epoch 20 Batch 500 Loss 0.9528 Accuracy 0.6733
Epoch 20 Batch 550 Loss 0.9536 Accuracy 0.6732


wandb: WARNING Step must only increase in log calls.  Step 20 < 21; dropping {'train_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.9535437>, 'train_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.67328715>}.


Saving checkpoint for epoch 20 at ./checkpoints/train/ckpt-8
Epoch 20 Loss 0.9535 Accuracy 0.6733
Time taken for 1 epoch: 111.53292512893677 secs

epoch lasted: 111.53683090209961
Epoch 21 Batch 0 Loss 0.9256 Accuracy 0.6970
Epoch 21 Batch 50 Loss 0.9464 Accuracy 0.6762
Epoch 21 Batch 100 Loss 0.9481 Accuracy 0.6753
Epoch 21 Batch 150 Loss 0.9489 Accuracy 0.6750
Epoch 21 Batch 200 Loss 0.9483 Accuracy 0.6755
discarded batch 247
Epoch 21 Batch 250 Loss 0.9500 Accuracy 0.6751
Epoch 21 Batch 300 Loss 0.9494 Accuracy 0.6752
Epoch 21 Batch 350 Loss 0.9493 Accuracy 0.6751
Epoch 21 Batch 400 Loss 0.9493 Accuracy 0.6751
Epoch 21 Batch 450 Loss 0.9493 Accuracy 0.6750
Epoch 21 Batch 500 Loss 0.9492 Accuracy 0.6751
Epoch 21 Batch 550 Loss 0.9489 Accuracy 0.6753
Epoch 21 Loss 0.9485 Accuracy 0.6754
Time taken for 1 epoch: 111.17384266853333 secs

discarded batch 5
Epoch 21 VALIDATION: Loss 0.9100 Accuracy 0.6879

epoch lasted: 111.5509307384491
Epoch 22 Batch 0 Loss 0.9889 Accuracy 0.6546
Epoch 22

In [15]:
#@title Parameter persistence

transformer.save_weights("./optimus_rhyme")
#transformer.load_weights("./optimus_rhyme")


#emb_enc_w = transformer.encoder.embedding.get_weights()[0]
emb_enc_w = transformer.decoder.embedding.get_weights()[0]
print(emb_enc_w.shape)

out_v = open('vecs.tsv', 'w', encoding='utf-8')
out_m = open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(poetry_sy_lm_dataset.vocabulary.dictionary):
  vec = emb_enc_w[num] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()


'''
%load_ext tensorboard
%tensorboard --logdir .
'''

(1900, 128)


'\n%load_ext tensorboard\n%tensorboard --logdir .\n'

# Tests

In [16]:
generate(1, k = 3, t = 0.9)

params: k=3, t=0.9
e | io | ch’ al | fi| ne | di | tut| t’ i | di| sii|
ap| pro| pin| qua| va | sì | com’ | io | do| vea|
l’ ar| dor | del | de| si| de| rio in | me | fi| nii|
                                          
 e | l’ al| tro | sie| gno| re e | la | suo | si | sta| schio|
e | l’ om| bia| te | ch’ in| na| to | son | per | la| gio|
e | la | sua | che | so| spen| te | so| sco| glia|
                            
 ch’ e’ | io | che ’l | for| ciò | di| co| le e | po| sci|
e | con | a| ma | che ’l | mio | con| ve| re | dio|
che | tu | di| si| ra | con | so| vi| ta| me | che| ra|
                                 
 che | l’ om| bi| glio a | li oc| chi | che i | fa| stros| so|
di | mi| so| se | se | la | ca| tu| ra | chia| scoc| ci|
con | l’ oc| cor | di | suo | so| lor | che | la | pas| so|
                    
 e | di| ca| tu| ra il | ma| la| va| ta in | si | por| ta|
ché | li oc| chi | con | l’ an| cor | si | sa| ra| to!|
e | i’ | se’ | co| mi| sciò ’l | pos| se| ne in | sua | fos| s

In [17]:
generate(1, k = 3, t = 1)

params: k=3, t=1
e | io | ch’ al | fi| ne | di | tut| t’ i | di| sii|
ap| pro| pin| qua| va | sì | com’ | io | do| vea|
l’ ar| dor | del | de| si| de| rio in | me | fi| nii|
                                          
 co| me | si | più | di| cea | che | so| sti| mo| gli|
pen| si | che | si | so| le a | lui | se| ce| se | fi| co|
e | la | con| tri| ma | co| lui a | men| tra | co| sto|
                               
 del | ciel | co| me | se | li oc| chi | di | lun| go|
che | se | so| li| co| me | di | la| gria| co| me|
e | per | l’ al| tro al | con| te | la | mio | ca| lui | ca| so| sto|
                        
 co| sì | dal | con| d’ i’ | mia | sua | ch’ io | fu | pa| schia|
co| me | so| lor | di | la | sua | che in | chiu| so|
e | che ’l | mi| che | di i | fuor | di| cel | pa| li| zia|
                    
 e ’l | mio | di | sé | suo | che ’l | par| ti | ca| pos| si|
co| mi | ch’ ei | ciò | da | li | fos| si| ra| ver| si|
e | io | co| me | da| ve| re | che | si | fos| si|
          

In [18]:
generate(1, k = 5, t = 1)

params: k=5, t=1
e | io | ch’ al | fi| ne | di | tut| t’ i | di| sii|
ap| pro| pin| qua| va | sì | com’ | io | do| vea|
l’ ar| dor | del | de| si| de| rio in | me | fi| nii|
                                          
 che | di | cu| no | può | dal | suo | la | sco| co| gni?|
den| na| vi| tà | co| sì | che | di| man| tra | co| gliu| na|
sì | com’ | i’ | più | l’ ac| ciò | dal | di| com’ | al| ma|
                   
 se | che | pri| sti| rin | sì | ch’ el | mon| tro | col| le|
e | l’ al| lor | co| men| si | sot| tri e ’l | san| do|
e | che | si | pri| ma | co| me e | tu | di | si | fal| la|
                      
 e | la | scas| si | die| co| lo | di| ce | sue | chian| ta|
la | che | li oc| chi ’l | mi | col | si | pro| tel| la|
che | de’ | la | sco| tro a | ma | sia | se | lui | fa| co| glia|
                
 con| di| stre | che | si | pri| ta | ch’ i’ | ten | fio|
e| ran| no in | fa| cian | d’ in| no| stra| noc| co| sco|
che | la | per| che | lui | par| gi| so | sca| gnes| si|
      